In [1]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)

%load_ext autoreload
%autoreload 2


In [2]:
import pandas as pd
import numpy as np
import polars as pl

import matplotlib.pyplot as plt
import seaborn as sns

from src.data import make_dataset
from src.features import build_features
from src.visualization import visualize

sns.set_style()
plt.rcParams["figure.figsize"] = (12, 6)

HOSPITAL_A_ANLIZAR = 112103  # Este es el Torax

## 1. Obtencion de Métricas de Egresos Hospitalarios

En este apartado se quieren calculcar diversas métricas relevantes para el análisis de los
egresos hospitalarios en Chile. Las métricas a calcular son las siguientes:

1. Cantidad de Egresos Hospitalarios
2. Dias de Estada Totales
3. Cantidad de Intervenciones Quirúrgicas
4. Cantidad de Muertos

Todas estas métricas serán calculadas para cada Año, Establecimiento de Salud y Diagnóstico
distinto que exista en la base de datos. Por ejemplo:

"En el año 2019, el Hospital San José (código XXXXXX) tuvo XX egresos, con XX dias estada promedio,
con XX intervenciones quirúrgicas y XX muertos para el diagnóstico A24.4"s


In [3]:
df_nacional = pl.scan_csv(
    "../data/processed/egresos_procesados.csv",
    infer_schema_length=10000,
    null_values="Extranjero",
)

In [4]:
AGRUPACION = [
    "ANO_EGRESO",
    "ESTABLECIMIENTO_SALUD",
    "GLOSA_ESTABLECIMIENTO_SALUD",
    "DIAG1",
]

metricas = build_features.obtener_metricas_egresos(df_nacional, AGRUPACION).collect()

In [5]:
display(metricas.sample(10))

ANO_EGRESO,ESTABLECIMIENTO_SALUD,GLOSA_ESTABLECIMIENTO_SALUD,DIAG1,n_egresos,dias_estada_totales,n_int_q,n_muertos,n_pacientes_distintos
i64,i64,str,str,u32,i64,i64,i64,u32
2009,115222,"""Clínica de Sal…","""A049""",5,15,0,0,5
2001,117202,"""Consultorio de…","""M236""",1,2,1,0,1
2002,119100,"""Hospital Las H…","""H401""",7,29,6,0,7
2004,105104,"""Hospital de Sa…","""K805""",7,12,0,0,7
2019,112211,"""Clínica Indisa…","""D839""",1,1,0,0,1
2010,105100,"""Hospital San J…","""K420""",8,44,7,1,8
2009,126100,"""Hospital Clíni…","""P284""",6,27,0,0,6
2002,116103,"""Hospital de Hu…","""N10X""",12,48,0,0,2
2018,118100,"""Hospital Clíni…","""R18X""",7,11,0,0,6


- Al ver una muestra del calculo realizado, se puede observar la cantidad de egresos, dias de
  estada, numero de intervenciones quirurgicas y cantidad de muertos para 10 duplas de
  hospitales - diagnosticos distintos.


Si bien se tienen las metricas calculadas, ahora se quiere responder a las siguientes preguntas:

- En qué diagnósticos se destaca cada hospital productivamente? Esto significa, en qué diagnósticos
  cada hospital tiene una mayor cantidad de egresos que otros hospitales?

- El hospital tiene un desempeño similar dentro de distintos grupos/estratos de hospitales?
  El hospital se desempeñara de la misma forma nacionalmente, dentro del estrato público, privado,
  etc...?

Para responder a la pregunta se realizará un ranking de todos los hospitales para cada
diagnóstico y cada año observado. Este ranking se realizará para distintos grupos/estratos de
hospitales (Nacional, Hospitales Públicos, Hospitales Privados, Hospitales GRD)


## 2. Análisis de Estratos

En este análisis se quiere observar el ranking de cada hospital/clínica del país para cada 
año y diagnóstico observado. Además, el ranking se realizará en los siguientes estratos:

1. Nacional
2. Públicos
3. Privados
4. Adscritos a GRD
5. INT

In [6]:
dict_estratos = build_features.obtener_diccionario_estratos(df_nacional, HOSPITAL_A_ANLIZAR)

In [7]:
variables_a_rankear = ["n_egresos"]
subgrupo_del_ranking = ["ANO_EGRESO", "DIAG1"]

rankings_nacionales = build_features.agregar_ranking_estratos(
    metricas, dict_estratos, variables_a_rankear, subgrupo_del_ranking
)

rankings_hospital_analizado = rankings_nacionales.filter(
    pl.col("ESTABLECIMIENTO_SALUD") == HOSPITAL_A_ANLIZAR
)

In [8]:
display(rankings_nacionales.head(8))

ANO_EGRESO,ESTABLECIMIENTO_SALUD,GLOSA_ESTABLECIMIENTO_SALUD,Capítulo,Sección,Categoría,Descripción,DIAG1,n_egresos,dias_estada_totales,n_int_q,n_muertos,n_pacientes_distintos,ranking_nacionales_n_egresos,total_nacionales_n_egresos,%_nacionales_n_egresos,ranking_publicos_n_egresos,total_publicos_n_egresos,%_publicos_n_egresos,ranking_privados_n_egresos,total_privados_n_egresos,%_privados_n_egresos,ranking_grd_n_egresos,total_grd_n_egresos,%_grd_n_egresos,ranking_interno_n_egresos,total_interno_n_egresos,%_interno_n_egresos
i64,i64,str,str,str,str,str,str,u32,i64,i64,i64,u32,u32,u32,f64,u32,u32,f64,u32,u32,f64,u32,u32,f64,u32,u32,f64
2020,104103,"""Hospital Provi…","""Cap.21 FACTOR…","""Z69-Z76 PERSO…","""Z76 PERSONAS E…","""Personas en co…","""Z769""",6,32,2,0,6,1,15,0.4,1,14,0.428571,null,null,null,1,6,1.0,null,null,null
2020,125102,"""Hospital Dr. L…","""Cap.21 FACTOR…","""Z69-Z76 PERSO…","""Z76 PERSONAS E…","""Personas en co…","""Z769""",3,8,0,0,3,2,15,0.2,2,14,0.214286,null,null,null,null,null,null,null,null,null
2020,125103,"""Hospital Lord …","""Cap.21 FACTOR…","""Z69-Z76 PERSO…","""Z76 PERSONAS E…","""Personas en co…","""Z769""",2,349,0,0,2,3,15,0.133333,3,14,0.142857,null,null,null,null,null,null,null,null,null
2020,102200,"""Clínica Iquiqu…","""Cap.21 FACTOR…","""Z69-Z76 PERSO…","""Z76 PERSONAS E…","""Personas en co…","""Z769""",1,3,1,0,1,4,15,0.066667,null,null,null,1,1,1.0,null,null,null,null,null,null
2020,118106,"""Hospital de Lo…","""Cap.21 FACTOR…","""Z69-Z76 PERSO…","""Z76 PERSONAS E…","""Personas en co…","""Z769""",1,1,0,0,1,4,15,0.066667,4,14,0.071429,null,null,null,null,null,null,null,null,null
2020,116101,"""Hospital de Te…","""Cap.21 FACTOR…","""Z69-Z76 PERSO…","""Z76 PERSONAS E…","""Personas en co…","""Z769""",1,1,0,0,1,4,15,0.066667,4,14,0.071429,null,null,null,null,null,null,null,null,null
2020,107107,"""Hospital de Pe…","""Cap.21 FACTOR…","""Z69-Z76 PERSO…","""Z76 PERSONAS E…","""Personas en co…","""Z769""",1,140,0,0,1,4,15,0.066667,4,14,0.071429,null,null,null,null,null,null,null,null,null
2020,112211,"""Clínica Indisa…","""Cap.21 FACTOR…","""Z69-Z76 PERSO…","""Z76 PERSONAS E…","""Persona en con…","""Z768""",58,118,14,1,57,1,299,0.19398,null,null,null,1,149,0.389262,null,null,null,null,null,null


Por lo tanto, se pueden observar todas las métricas de egresos hospitalarios nacionales para
cada estrato definido. Además, se observa que en casos de empate en cantidad de egresos,
el ranking es repetido.

## 3. Diagnósticos más relevantes del Instituto Nacional del Tórax

En este apartado se quieren responder las siguientes preguntas:

1. ¿Cuáles son los **diagnósticos más relevantes productivamente** en el Instituto Nacional del
Tórax?
2. ¿En qué diagnósticos el INT tiene una mayor cantidad de Egresos que cualquier otro establecimiento
de salud Nacional? En otras palabras, ¿En qué diagnósticos el **INT tiene un Ranking 1 Nacional**?

Estas preguntas se quieren responder tanto para diagnósticos ambulatorios (n_int_q = 0) y 
quirúrgicos (n_int_q > 0).

Además, se sabe que desde el 2011 el registro de Egresos Hospitalarios DEIS fue llevado por la
unidad de Grupo Relacionado al Diagnóstico (GRD). Debido a lo anterior, la calidad de la codificación
de diagnósticos es mucho mayor. Tomado de lo anterior, **se obtendrán los diagnósticos que tengan
una alta productividad desde el 2012 hasta 2020**.

In [9]:
DIAGS_MAS_RELEVANTES_QUIRURGICOS = rankings_hospital_analizado.filter(
    (pl.col("ranking_nacionales_n_egresos") == 1)
    & (pl.col("n_int_q") > 0)
    & (pl.col("ANO_EGRESO") > 2011)
).sort(by=["ANO_EGRESO", "n_egresos", "n_int_q"], descending=True)


# Obtiene los 20 diagnosticos con la mayor cantidad de egresos e int_q
veinte_mas_relevantes_por_anio_quir = DIAGS_MAS_RELEVANTES_QUIRURGICOS.group_by(
    ["ANO_EGRESO"]
).head(50)

In [10]:
DIAGS_MAS_RELEVANTES_QUIRURGICOS = veinte_mas_relevantes_por_anio_quir.select(
    pl.col("DIAG1").unique()
).sort(by="DIAG1")

In [11]:
print(list(DIAGS_MAS_RELEVANTES_QUIRURGICOS.to_series()))
print(f"Hay {len(DIAGS_MAS_RELEVANTES_QUIRURGICOS)} diagnosticos quirurgicos altamente "
      f"relevantes para el país")

['A152', 'A156', 'A157', 'B334', 'B441', 'B671', 'C33X', 'C340', 'C341', 'C342', 'C343', 'C37X', 'C381', 'C382', 'C383', 'C384', 'C413', 'C450', 'C457', 'C761', 'C780', 'C781', 'C782', 'C797', 'D142', 'D143', 'D144', 'D150', 'D151', 'D152', 'D157', 'D167', 'D174', 'D381', 'D382', 'D383', 'D860', 'D861', 'D862', 'E848', 'G580', 'G908', 'I011', 'I051', 'I052', 'I060', 'I062', 'I071', 'I080', 'I081', 'I232', 'I233', 'I250', 'I272', 'I311', 'I330', 'I340', 'I350', 'I351', 'I352', 'I360', 'I361', 'I371', 'I372', 'I420', 'I421', 'I422', 'I425', 'I441', 'I453', 'I456', 'I472', 'I495', 'I513', 'I710', 'I711', 'I712', 'I715', 'I716', 'J151', 'J398', 'J439', 'J47X', 'J679', 'J840', 'J841', 'J848', 'J849', 'J850', 'J851', 'J852', 'J860', 'J869', 'J90X', 'J91X', 'J929', 'J931', 'J938', 'J940', 'J955', 'J958', 'J980', 'J986', 'L758', 'M051', 'M348', 'Q211', 'Q222', 'Q225', 'Q228', 'Q231', 'Q233', 'Q256', 'Q257', 'Q676', 'Q677', 'Q678', 'Q766', 'R048', 'S225', 'T820', 'T821', 'T822', 'T862', 'T863',

In [12]:
DIAGS_MAS_RELEVANTES_AMBULATORIOS = rankings_hospital_analizado.filter(
    (pl.col("ranking_nacionales_n_egresos") == 1)
    & (pl.col("n_int_q") == 0)
    & (pl.col("ANO_EGRESO") > 2011)
).sort(by=["ANO_EGRESO", "n_egresos", "n_int_q"], descending=True)

veinte_mas_relevantes_por_anio_ambul = DIAGS_MAS_RELEVANTES_AMBULATORIOS.group_by(
    ["ANO_EGRESO"]
).head(20)

DIAGS_MAS_RELEVANTES_AMBULATORIOS = veinte_mas_relevantes_por_anio_ambul.select(
    pl.col("DIAG1").unique()
).sort(by="DIAG1")

print(list(DIAGS_MAS_RELEVANTES_AMBULATORIOS.to_series()))
print(f"Hay {len(DIAGS_MAS_RELEVANTES_AMBULATORIOS)} diagnosticos ambulatorios altamente "
      f"relevantes para el país")

['A022', 'A151', 'A152', 'A154', 'A155', 'A157', 'A190', 'B371', 'B441', 'B460', 'B59X', 'B948', 'C33X', 'C382', 'C783', 'D611', 'D860', 'D862', 'E662', 'E854', 'G580', 'I062', 'I082', 'I150', 'I232', 'I253', 'I270', 'I279', 'I280', 'I421', 'I422', 'I424', 'I427', 'I445', 'I447', 'I451', 'I456', 'I458', 'I471', 'I472', 'I483', 'I493', 'I494', 'I513', 'I715', 'I716', 'J150', 'J155', 'J430', 'J441', 'J632', 'J677', 'J678', 'J679', 'J684', 'J700', 'J708', 'J82X', 'J840', 'J848', 'J940', 'J951', 'J958', 'L581', 'M051', 'M348', 'M350', 'M358', 'M485', 'M941', 'Q218', 'Q222', 'Q225', 'Q228', 'Q233', 'Q240', 'Q241', 'Q257', 'Q258', 'Q874', 'R048', 'S203', 'T366', 'T371', 'T380', 'T822', 'T862', 'T863', 'T868', 'Z030', 'Z100', 'Z118', 'Z136', 'Z291', 'Z292', 'Z488', 'Z514', 'Z516', 'Z541', 'Z748', 'Z851', 'Z858', 'Z877', 'Z911', 'Z924', 'Z952', 'Z953', 'Z998']
Hay 108 diagnosticos ambulatorios altamente relevantes para el país


Por lo tanto, se reportan los diagnósticos altamente relevantes para el país productivamente
hablando. En el ámbito quirúrgico, existen 45 diagnósticos dónde el INT tiene la mayor cantidad
de Egresos Hospitalarios del país entre 2012 y 2020. En el ámbito "ambulatorio" (egresos sin int_q),
existen 108 diagnósticos con una alta producción a nivel país.

Para el MINSAL, se quiere reportar los diagnósticos quirúrgicos que hayan tenido un ranking 1 o 2 
en el año 2019 para el INT.

In [13]:
DIAGS_1_O_2_EN_2019 = rankings_hospital_analizado.filter(
    (pl.col("ANO_EGRESO") == 2019)
    & (
        (pl.col("ranking_nacionales_n_egresos") == 1)
        | (pl.col("ranking_nacionales_n_egresos") == 2)
    )
    & (pl.col("DIAG1").is_in(list(DIAGS_MAS_RELEVANTES_QUIRURGICOS.to_series())))
).select(pl.col("DIAG1").unique()).sort("DIAG1")

print(list(DIAGS_1_O_2_EN_2019.to_series()))
print(
    f"Hay {len(DIAGS_1_O_2_EN_2019)} diagnosticos relevantes productivamente Y que "
    f"tenian ranking 1 o 2 en 2019"
)

['A152', 'A156', 'B441', 'B671', 'C33X', 'C340', 'C341', 'C342', 'C343', 'C381', 'C382', 'C384', 'C450', 'C457', 'C780', 'C782', 'D142', 'D143', 'D150', 'D151', 'D152', 'D157', 'D174', 'D381', 'D382', 'D383', 'D860', 'D862', 'E848', 'G908', 'I051', 'I052', 'I080', 'I081', 'I232', 'I311', 'I330', 'I340', 'I351', 'I352', 'I361', 'I420', 'I421', 'I453', 'I456', 'I472', 'I495', 'I513', 'I710', 'I711', 'I712', 'I715', 'I716', 'J47X', 'J679', 'J841', 'J848', 'J849', 'J851', 'J860', 'J869', 'J929', 'J955', 'J986', 'M051', 'M348', 'Q222', 'Q233', 'Q257', 'Q677', 'Q766', 'R048', 'T820', 'T821', 'T822', 'T868', 'Z450']
Hay 77 diagnosticos relevantes productivamente Y que tenian ranking 1 o 2 en 2019


In [37]:
tabla_resumen_egresos_torax_relevantes = (
    rankings_hospital_analizado.filter(
        ((pl.col("ANO_EGRESO") >= 2016) & (pl.col("ANO_EGRESO") <= 2020))
        & (pl.col("DIAG1").is_in(list(DIAGS_1_O_2_EN_2019.to_series())))
    )
    .with_columns((pl.col("DIAG1") + " - " + pl.col("Descripción")).alias("Diagnóstico"))
    .to_pandas()
)

tabla_resumen_egresos_torax_relevantes.to_excel(
    "../data/interim/resumen_diagnosticos_relevantes_INT_largo.xlsx"
)

In [38]:
tabla_resumen_egresos_torax_relevantes = pd.pivot_table(
    tabla_resumen_egresos_torax_relevantes,
    index=["Diagnóstico"],
    columns="ANO_EGRESO",
    values=[
        "ranking_nacionales_n_egresos",
        "n_pacientes_distintos",
        "n_egresos",
        "total_nacionales_n_egresos",
        "%_nacionales_n_egresos",
        "dias_estada_totales",
    ],
    aggfunc="sum",
    fill_value=0,
).sort_values(
    by=[("ranking_nacionales_n_egresos", 2019), ("n_egresos", 2019)],
    ascending=[True, False],
)[
    [
        "dias_estada_totales",
        "n_pacientes_distintos",
        "n_egresos",
        "total_nacionales_n_egresos",
        "%_nacionales_n_egresos",
        "ranking_nacionales_n_egresos",
    ]
]

tabla_resumen_egresos_torax_relevantes.to_excel(
    "../data/interim/resumen_diagnosticos_relevantes_INT.xlsx"
)

In [39]:
resumen_egresos_torax_y_nacional_relevantes = (
    tabla_resumen_egresos_torax_relevantes[("n_egresos")].astype(int).astype(str)
    + " ("
    + tabla_resumen_egresos_torax_relevantes[("total_nacionales_n_egresos")].astype(int).astype(str)
    + "; "
    + round((tabla_resumen_egresos_torax_relevantes[("%_nacionales_n_egresos")] * 100), 1).astype(
        str
    )
    + "%)"
)

posiciones_torax_egresos_relevantes = tabla_resumen_egresos_torax_relevantes[
    ("ranking_nacionales_n_egresos")
].astype(int)

dias_estada_egresos_relevantes = tabla_resumen_egresos_torax_relevantes[
    ("dias_estada_totales")
].astype(int)

cantidad_camas_requeridas = round(dias_estada_egresos_relevantes / 365, 1)

resumen_egresos_y_posiciones_relevantes = pd.concat(
    [
        resumen_egresos_torax_y_nacional_relevantes,
        posiciones_torax_egresos_relevantes,
        dias_estada_egresos_relevantes,
        cantidad_camas_requeridas,
    ],
    axis=1,
)

resumen_egresos_y_posiciones_relevantes.columns = [
    "2016",
    "2017",
    "2018",
    "2019",
    "2016",
    "2017",
    "2018",
    "2019",
    "2016",
    "2017",
    "2018",
    "2019",
    "2016",
    "2017",
    "2018",
    "2019",
]

display(resumen_egresos_y_posiciones_relevantes)

ValueError: Length mismatch: Expected axis has 20 elements, new values have 16 elements

In [56]:
resumen_egresos_y_posiciones_relevantes = resumen_egresos_y_posiciones_relevantes.reset_index()

In [57]:
visualize.add_dataframes_to_powerpoint(
    [resumen_egresos_y_posiciones_relevantes],
    "../reports/presentacion.pptx",
    font_size=8,
    cell_width=33,
    cell_height=0.5,
    font_family="Open Sans",
    max_cell_characters=100,
)

Para hacer un contorl de lo que se calculo, se observará una muestra de 10 hospitales - diagnósticos
distintos


In [ ]:
display(rankings_nacionales.sample(10))

ANO_EGRESO,ESTABLECIMIENTO_SALUD,GLOSA_ESTABLECIMIENTO_SALUD,Capítulo,Sección,Categoría,Descripción,DIAG1,n_egresos,dias_estada_totales,n_int_q,n_muertos,n_pacientes_distintos,ranking_nacionales_n_egresos,total_nacionales_n_egresos,%_nacionales_n_egresos,ranking_publicos_n_egresos,total_publicos_n_egresos,%_publicos_n_egresos,ranking_privados_n_egresos,total_privados_n_egresos,%_privados_n_egresos,ranking_grd_n_egresos,total_grd_n_egresos,%_grd_n_egresos,ranking_interno_n_egresos,total_interno_n_egresos,%_interno_n_egresos
i64,i64,str,str,str,str,str,str,u32,i64,i64,i64,u32,u32,u32,f64,u32,u32,f64,u32,u32,f64,u32,u32,f64,u32,u32,f64
2013,123100,"""Hospital Base …","""Cap.06 ENFERM…","""G40-G47 TRAST…","""G44 OTROS SÍND…","""Síndrome de ce…","""G440""",1,1,0,0,1,10,64,0.015625,5,23,0.043478,null,null,null,3,12,0.083333,null,null,null
2001,104104,"""Hospital Dr. M…","""Cap.19 LESION…","""S30-S39 TRAUM…","""S30 TRAUMATISM…","""Contusión de l…","""S300""",1,1,0,0,1,155,963,0.001038,100,526,0.001901,null,null,null,null,null,null,null,null,null
2016,115106,"""Hospital de Pi…","""Cap.05 TRASTO…","""F10-F19 TRAST…","""F10 TRASTORNOS…","""Trastornos men…","""F103""",2,14,0,0,2,72,618,0.003236,71,601,0.003328,null,null,null,null,null,null,null,null,null
2006,112100,"""Hospital Del S…","""Cap.13 ENFERM…","""M20-M25 OTROS…","""M23 TRASTORNO …","""Trastorno inte…","""M239""",4,4,4,0,4,31,1152,0.003472,8,136,0.029412,null,null,null,8,127,0.031496,null,null,null
2008,114103,"""Hospital Padre…","""Cap.15 EMBARA…","""O85-O92 COMPL…","""O90 COMPLICACI…","""Otras complica…","""O908""",1,17,1,0,1,37,544,0.001838,32,521,0.001919,null,null,null,16,296,0.003378,null,null,null
2016,116101,"""Hospital de Te…","""Cap.03 ENFERM…","""D55-D59 ANEMI…","""D59 ANEMIA HEM…","""Anemia hemolít…","""D599""",2,6,0,0,1,3,64,0.03125,3,54,0.037037,null,null,null,null,null,null,null,null,null
2010,112211,"""Clínica Indisa…","""Cap.16 CIERTA…","""P70-P75 TRAST…","""P70 TRASTORNOS…","""Otras hipoglic…","""P704""",7,81,0,0,7,35,1348,0.005193,null,null,null,8,267,0.026217,null,null,null,null,null,null
2001,110100,"""Hospital San J…","""Cap.04 ENFERM…","""E70-E88 ALTER…","""E87 OTROS TRAS…","""Hiperpotasemia…","""E875""",1,1,0,1,1,7,31,0.032258,6,25,0.04,null,null,null,6,21,0.047619,null,null,null
2017,115110,"""Hospital de Sa…","""Cap.01 CIERTA…","""A00-A09 ENFER…","""A08 INFECCIONE…","""Infección inte…","""A084""",2,9,0,0,2,42,370,0.005405,32,265,0.007547,null,null,null,22,172,0.011628,null,null,null


En la muestra anterior se puede observar las métricas previamente calculada, y el ranking
respectivo que tiene cada hospital para diagnósticos en cada año específico.


In [ ]:
rankings_nacionales.to_pandas().to_csv(
    "../data/interim/ranking_nacional_egresos.csv",
    sep=";",
    decimal=".",
    encoding="latin-1",
    index=False,
)

rankings_hospital_analizado.to_pandas().to_csv(
    "../data/interim/ranking_torax_egresos.csv",
    sep=";",
    decimal=".",
    encoding="latin-1",
    index=False,
)

## Metricas para personas sobre 14 años

En este análisis se quiere ver el ranking de egresos pero solamente tomando en cuenta a personas
mayores a 14 años. Esto, para dilucidar si el hospital del tórax es altamente relevante para tratar
patologías congénitas de adultos.


In [ ]:
df_nacional_mayores_a_14 = df_nacional.filter(pl.col("EDAD_A_OS") > 14)

In [ ]:
metricas_sobre_14 = build_features.obtener_metricas_egresos(
    df_nacional_mayores_a_14, AGRUPACION
).collect()

In [ ]:
variables_a_rankear = ["n_egresos"]
subgrupo_del_ranking = ["ANO_EGRESO", "DIAG1"]

rankings_nacionales_sobre_14 = build_features.agregar_ranking_estratos(
    metricas_sobre_14, dict_estratos, variables_a_rankear, subgrupo_del_ranking
)

diags_congenitos_relevantes = rankings_nacionales_sobre_14.filter(
    (pl.col("DIAG1").is_in(["Q230", "Q211", "Q231"])) & (pl.col("ANO_EGRESO") == 2019)
)

In [ ]:
diags_congenitos_relevantes.to_pandas().to_csv(
    "../data/interim/diags_congenitas_sobre_14_egresos.csv",
    sep=";",
    decimal=".",
    encoding="latin-1",
    index=False,
)

## 3. Análisis de procedimientos DEIS

En este análisis se quieren responder las siguientes preguntas:

1. ¿Cuál es el procedimiento más frecuente dentro de los pacientes que se les realizó un procedimiento
   quirúrgico?


In [ ]:
proced = (
    df_nacional.filter(pl.col("INTERV_Q") == 1)
    .groupby(["ANO_EGRESO", "ESTABLECIMIENTO_SALUD", "GLOSA_PROCED_PPAL"])
    .agg(pl.count())
).collect()

/var/folders/wy/2slj_t7s2vz19v2h2926qb340000gn/T/ipykernel_24941/3397463805.py:3: DeprecationWarning: `groupby` is deprecated. It has been renamed to `group_by`.
  .groupby(["ANO_EGRESO", "ESTABLECIMIENTO_SALUD", "GLOSA_PROCED_PPAL"])


In [ ]:
display(proced)

ANO_EGRESO,ESTABLECIMIENTO_SALUD,GLOSA_PROCED_PPAL,count
i64,i64,str,u32
2001,112100,null,13867
2001,101100,null,4816
2001,112103,null,1143
2001,109101,null,4618
2001,115100,null,7926
2001,121202,null,3181
2001,120101,null,3313
2001,118200,null,1996
2001,120105,null,60


Los resultados muestran el desglose por año y recinto hospitalario junto a la cantidad de
procedimientos realizados en cada intervención quirúrgica. Sin embargo, se puede observar que
existe una cantidad importante de casos donde la glosa del procedimiento fue nula.

Ahora, se quiere calcular cuántas intervenciones quirúrgicas carecen de una glosa de procedimiento.


In [ ]:
display(proced.groupby("GLOSA_PROCED_PPAL").agg(pl.col("count").sum()).sort("count"))

/var/folders/wy/2slj_t7s2vz19v2h2926qb340000gn/T/ipykernel_24941/989951252.py:1: DeprecationWarning: `groupby` is deprecated. It has been renamed to `group_by`.
  display(proced.groupby("GLOSA_PROCED_PPAL").agg(pl.col("count").sum()).sort("count"))


GLOSA_PROCED_PPAL,count
str,u32
"""Cuerpo extraño…",1
"""Tomografía Com…",1
"""Dermatoscopia …",1
"""Electromiograf…",1
"""Calzon corto d…",1
"""Rinomanometria…",1
"""Función tubari…",1
"""Sesión de Psic…",1
"""Volúmenes pulm…",1


Por lo tanto, 12382642 de todas las intervenciones quirúrgicas carecen de una glosa de procedimiento.


In [ ]:
proced.write_excel("../data/interim/conteo_procedimientos.xlsx")

# 4. Obtencion de Metricas por Meses

En este apartado se quieren obtener las metricas de Egresos (n_egresos, dias_estada, n_int_q, n_muertos)
por Meses.

In [ ]:
df_nacional_fecha = df_nacional.with_columns(pl.col("FECHA_EGRESO").str.to_date()).with_columns(
    [
        pl.col("FECHA_EGRESO").dt.month().alias("MES_EGRESO"),
        pl.col("FECHA_EGRESO").dt.day().alias("DIA_EGRESO"),
    ]
)

In [ ]:
AGRUPACION_POR_MES_Y_DIA = [
    "ANO_EGRESO",
    "MES_EGRESO",
    "DIA_EGRESO",
    "ESTABLECIMIENTO_SALUD",
    "GLOSA_ESTABLECIMIENTO_SALUD",
    "DIAG1",
]

metricas_por_mes_y_dia = build_features.obtener_metricas_egresos(
    df_nacional_fecha, AGRUPACION_POR_MES_Y_DIA
).collect()

In [ ]:
metricas_torax_por_mes_y_dia = metricas_por_mes_y_dia.filter(
    pl.col("ESTABLECIMIENTO_SALUD") == HOSPITAL_A_ANLIZAR
)

In [ ]:
metricas_torax_por_mes_y_dia.write_csv("../data/interim/egresos_torax_mes_y_dia.csv")

In [ ]:
resumen_cancer_pulmon_nacional = (
    df_nacional.filter(pl.col("DIAG1").str.contains("C34"))
    .group_by("ANO_EGRESO")
    .agg(
        [
            pl.col("DIAG1").count().alias("n_egresos"),
            pl.col("DIAS_ESTADA").sum().alias("dias_estada_totales"),
            pl.col("INTERV_Q").sum().alias("n_int_q"),
            pl.col("CONDICION_EGRESO").sum().alias("n_muertos"),
            pl.col("ID_PACIENTE").n_unique().alias("n_pacientes_distintos"),
        ]
    )
).collect()

In [ ]:
resumen_cancer_pulmon_nacional.sort("ANO_EGRESO")

ANO_EGRESO,n_egresos,dias_estada_totales,n_int_q,n_muertos,n_pacientes_distintos
i64,u32,i64,i64,i64,u32
2001,2103,21396,384,363,1119
2002,2797,25587,424,430,1342
2003,3359,31451,504,384,1806
2004,3415,29460,287,418,1592
2005,3820,29303,394,427,1934
2006,3394,28819,433,434,1550
2007,3345,29894,484,422,1771
2008,3066,28539,375,412,1783
2009,3276,28272,520,429,1943
